In [3]:
import pprint

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

In [5]:
from paths import *

In [6]:
options = Options()
options.binary_location = browser_path
options.add_argument(f'--profile {profile_path}')
driver = webdriver.Firefox(service=Service(driver_path), options=options)

In [23]:
char_whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
def sanitize_name(name):
    return (''.join(filter(char_whitelist.__contains__, name))).upper()

In [24]:
def reset_state():
    global names_to_ids
    global building_ids_to_names
    names_to_ids = {}
    building_ids_to_names = {}

In [25]:
def register_name_id(name, id):
    global names_to_ids
    names_to_ids[name] = id

In [26]:
def find_wonders(can_have_built):
    revealed_wonders = set()
    selected_wonders = set()
    built_wonders = set()

    for wonder in driver.find_elements(By.CLASS_NAME, 'wonder'):
        wonder_name = sanitize_name(wonder.text)
        wonder_id = wonder.get_attribute('id')

        register_name_id(wonder_name, wonder_id)

        constructed = int(wonder.get_attribute('data-constructed'))
        container = wonder.find_element(By.XPATH, "../../../..")

        if container.get_attribute('id') == 'wonder_selection_block':
            revealed_wonders.add(wonder_name)
        elif constructed:
            built_wonders.add(wonder_name)
        else:
            selected_wonders.add(wonder_name)

    if len(revealed_wonders) > 0:
        selected_wonders |= built_wonders
        built_wonders = set()

    return revealed_wonders, selected_wonders, built_wonders

In [27]:
def find_cards():
    global building_ids_to_names

    pyramid_poses_cards = set()
    guild_poses = set()
    built_cards = set()
    discarded_cards = set()

    for card in driver.find_elements(By.CLASS_NAME, 'building'):
        card_name = sanitize_name(card.text)
        card_id = card.get_attribute('id')
        card_bulding_id = card.get_attribute('data-building-id')

        if card_bulding_id == '' or card_bulding_id is None:
            back_pos = card.value_of_css_property('background-position')
            if back_pos == '-100% -700%':
                position = int(card.get_attribute('data-location'))
                guild_poses.add(position)
            continue

        if card_name != '':
            building_ids_to_names[card_bulding_id] = card_name
        else:
            card_name = building_ids_to_names[card_bulding_id]

        register_name_id(card_name, card_id)

        container = card.find_element(By.XPATH, "../..")
        container_id = container.get_attribute('id')

        if container_id == 'draftpool_container':
            position = int(card.get_attribute('data-location'))
            pyramid_poses_cards.add((position, card_name))
            if 'GUILD' in card_name:
                guild_poses.add(position)
        elif container_id == 'discarded_cards_container':
            discarded_cards.add(card_name)
        else:
            built_cards.add(card_name)

    return pyramid_poses_cards, guild_poses, built_cards, discarded_cards


In [28]:
def find_tokens():
    game_tokens = set()
    box_tokens = set()
    built_tokens = set()

    for token in driver.find_elements(By.CLASS_NAME, 'progress_token'):
        token_name = sanitize_name(token.text)
        token_id = token.get_attribute('id')

        register_name_id(token_name, token_id)

        container = token.find_element(By.XPATH, "../..")
        container_id = container.get_attribute('id')

        if container_id == 'board_progress_tokens':
            game_tokens.add(token_name)
        elif container_id == 'progress_token_from_box_container':
            box_tokens.add(token_name)
        else:
            built_tokens.add(token_name)

    return game_tokens, box_tokens, built_tokens

In [29]:
def select_by_id(id):
    WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.ID, id))).click()

def select_by_name(name):
    id = names_to_ids[name]
    select_by_id(id)

def open_bga_page():
    driver.get("https://boardgamearena.com/lobby")

def create_game():
    select_by_id('joingame_create_1266')

def open_table():
    select_by_id('open_table_now')

def accept_game():
    select_by_id('ags_start_game_accept')

def discard_card(name):
    select_by_name(name)
    select_by_id('buttonDiscardBuilding')

def build_card(name):
    select_by_name(name)
    select_by_id('buttonConstructBuilding')

def build_wonder(name, wonder_name):
    select_by_name(name)
    select_by_id('buttonConstructWonder')
    select_by_name(wonder_name)

def choose_go_first():
    select_by_id('buttonPlayerLeft')

def choose_go_second():
    select_by_id('buttonPlayerRight')

In [32]:
open_bga_page()
create_game()
open_table()
accept_game()

In [303]:
can_have_built = False

old_everything = []

while True:
    try:
        game_tokens, box_tokens, built_tokens = find_tokens()
        pyramid_poses_cards, guild_poses, built_cards, discarded_cards = find_cards()
        can_have_built |= len(pyramid_poses_cards) > 0
        revealed_wonders, selected_wonders, built_wonders = find_wonders(can_have_built)

        everything = [game_tokens, box_tokens, built_tokens, pyramid_poses_cards, guild_poses, built_cards, discarded_cards, revealed_wonders, selected_wonders, built_wonders]

        if everything != old_everything:
            old_everything = everything

            pprint.pprint(everything)
            print()
    except Exception:
        pass

[{'ARCHITECTURE', 'THEOLOGY', 'STRATEGY', 'URBANISM', 'AGRICULTURE'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'CIRCUS MAXIMUS', 'THE STATUE OF ZEUS', 'THE PYRAMIDS'},
 {'THE GREAT LIBRARY'},
 set()]

[{'ARCHITECTURE', 'THEOLOGY', 'STRATEGY', 'URBANISM', 'AGRICULTURE'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'THE STATUE OF ZEUS', 'THE PYRAMIDS'},
 {'CIRCUS MAXIMUS', 'THE GREAT LIBRARY'},
 set()]

[{'ARCHITECTURE', 'THEOLOGY', 'STRATEGY', 'URBANISM', 'AGRICULTURE'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'THE PYRAMIDS'},
 {'CIRCUS MAXIMUS', 'THE STATUE OF ZEUS', 'THE GREAT LIBRARY'},
 set()]

[{'ARCHITECTURE', 'THEOLOGY', 'STRATEGY', 'URBANISM', 'AGRICULTURE'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'CIRCUS MAXIMUS', 'THE STATUE OF ZEUS', 'THE PYRAMIDS', 'THE GREAT LIBRARY'}]

[{'ARCHITECTURE', 'THEOLOGY', 'STRATEGY', 'URBANISM', 'AGRICULTURE'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'', 'PIRAEUS', 'THE TEMPLE OF ARTEM